# Mesh plane and line

It is sometimes necessary to get the coordinates of points, which are equally spaced on a line. Those values can then be used for sampling fields. In `discretisedfield`, this can be obtained using `line`.

`line` takes 3 input arguments:

- points `p1` and `p2` between which the line spans,
- the number of points on the line `n`.

The mesh we are going to use as an example is:

In [1]:
import discretisedfield as df

p1 = (0, 0, 0)
p2 = (100e-9, 50e-9, 20e-9)
n = (20, 10, 4)

region = df.Region(p1=p1, p2=p2)
mesh = df.Mesh(region=region, n=n)

Let us say we want to get the coordinates of 11 (`n=11`) points on the line, which spans between $(0, 0, 0)$ and $(100\,\text{nm}, 0, 20\,\text{nm})$.

In [2]:
# NBVAL_IGNORE_OUTPUT
mesh.line(p1=(0, 0, 0), p2=(100e-9, 0, 20e-9), n=11)

<generator object Mesh.line at 0x7f856491c970>

This method returns a generator, which we can use as an iterator (for instance, in `for` loops). The coordinates of points are:

In [3]:
line = mesh.line(p1=(0, 0, 0), p2=(100e-9, 0, 20e-9), n=11)
list(line)

[(0.0, 0.0, 0.0),
 (1e-08, 0.0, 2e-09),
 (2e-08, 0.0, 4e-09),
 (3.0000000000000004e-08, 0.0, 6.000000000000001e-09),
 (4e-08, 0.0, 8e-09),
 (5e-08, 0.0, 1e-08),
 (6.000000000000001e-08, 0.0, 1.2000000000000002e-08),
 (7e-08, 0.0, 1.4000000000000001e-08),
 (8e-08, 0.0, 1.6e-08),
 (9e-08, 0.0, 1.8000000000000002e-08),
 (1e-07, 0.0, 2e-08)]

When asking for a mesh to give us the points on a line, both points `p1` and `p2` must belong to the mesh. For instance, if we ask for the following line:

In [4]:
p1 = (0, 0, 0)
p2 = (100e-9, 100e-9, 100e-9)

try:
    list(mesh.line(p1=p1, p2=p2, n=10))
except ValueError:
    print("Exception raised.")

Exception raised.


This is because point `p2` does not belong to the mesh region (`mesh.region`):

In [5]:
p2 in mesh.region

False

## Plane mesh

Similar to the line, we usually need to extract a plane-mesh. This method we later use for slicing fields and extracting the values of a field on the plane.

Plane mesh is obtained using `sel` method. The planes allowed must be perpendicular to the geometrical axes. Therefore, to extract a plane, we need to define the axis to which the plane is perpendicular to as well as the point at which the plane intersects the axis. For example, for a three-dimensional geometry, we want to extract the plane perpendicular to the z-axis (in the xy-plane), which intersects it at $2.5\,\text{nm}$:

In [6]:
mesh.sel(z=2.5e-9)

Mesh(Region(pmin=[0.0, 0.0], pmax=[1e-07, 5e-08], dims=['x', 'y'], units=['m', 'm']), n=[20, 10])

This method returns a mesh object, which consists no cells in the z-direction and keeps the same number of cells in x and y directions:

In [7]:
mesh.sel(z=2.5e-9).n

array([20, 10])

In other words, the discretisation `cell` in the plane mesh is actually two-dimensional with the same x and y dimensions of the `cell` in the original mesh.

The edge lengths of the resulting mesh region is:

In [8]:
mesh.sel(z=2.5e-9).region.edges

array([1.e-07, 5.e-08])

This means that the plane mesh keeps the original dimensions in x and y directions and has no "thickness".

Another way for asking for a plane mesh is simply by passing a string `'x'`, `'y'`, or `'z'`, without specifying the point. In that case, the mesh is sliced through the middle of the sample: 

In [9]:
plane_mesh = mesh.sel("x")

plane_mesh.region.centre

array([2.5e-08, 1.0e-08])

which is the same as the centre of the original mesh on the yz-plane:

In [10]:
mesh.region.centre[1:]

array([2.5e-08, 1.0e-08])